<a href="https://colab.research.google.com/github/Tiabet/DACON-code/blob/master/%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import lightgbm as lgbm
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, log_loss

In [3]:
# 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
train_df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


1. 대출 목적 비슷한 것들은 묶기
2. 근로기간 언노운 처리하고 1이하랑 1 묶기
3. 주택소유상태 ANY 처리
4. 연체계좌수, 총연체금액 드랍

In [5]:
train_df.drop(['연체계좌수', '총연체금액'], axis=1, inplace = True)

In [6]:
train_df.loc[train_df['주택소유상태']=='ANY','주택소유상태'] = 'MORTGAGE'

In [7]:
train_df['주택소유상태'].value_counts()

MORTGAGE    47935
RENT        37705
OWN         10654
Name: 주택소유상태, dtype: int64

In [8]:
train_df.loc[train_df['근로기간']=='3','근로기간']='3 years'
train_df.loc[train_df['근로기간']=='1 year','근로기간']='1 years'
train_df.loc[train_df['근로기간']=='<1 year','근로기간']='< 1 year'
train_df.loc[train_df['근로기간']=='10+years','근로기간']='10+ years'
train_df.value_counts('근로기간')

근로기간
10+ years    32481
2 years       8450
< 1 year      8144
3 years       7670
1 years       6305
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
dtype: int64

In [9]:
values_map=dict({'6 years':5,
 '10+ years':10,
 '5 years':1,
 '8 years':5,
 '9 years':5,
 '2 years':1,
 '1 years':1,
 '3 years':1,
 '7 years':5,
 '4 years':5,
 '< 1 year':1})

In [10]:
train_df.loc[train_df['근로기간'] == 'Unknown', '근로기간'] = np.random.choice([1,5,10], size=sum(train_df['근로기간'] == 'Unknown'))

In [11]:
train_df['근로기간'] = train_df['근로기간'].replace(values_map)

In [12]:
train_df.loc[train_df['근로기간']==6,'근로기간']= np.random.randint(1,3)

In [13]:
train_df['근로기간'].value_counts()

1     38121
10    34419
5     23754
Name: 근로기간, dtype: int64

In [14]:
train_df['대출목적'].value_counts()

부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4725
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: 대출목적, dtype: int64

In [15]:
train_df.loc[~train_df['대출목적'].isin(['부채 통합', '신용 카드', '기타']), '대출목적'] = '기타'

In [16]:
train_df['대출목적'].value_counts()

부채 통합    55150
신용 카드    24500
기타       16644
Name: 대출목적, dtype: int64

In [17]:
train_df.loc[train_df['대출기간']==' 36 months', '대출기간'] = 36
train_df.loc[train_df['대출기간']==' 60 months', '대출기간'] = 60

In [18]:
train_df.value_counts('대출기간')

대출기간
36    64479
60    31815
dtype: int64

In [19]:
train_df['최근_2년간_연체_횟수'] = np.where(train_df['최근_2년간_연체_횟수'] > 2, 2, train_df['최근_2년간_연체_횟수'])

In [20]:
train_df['대출기간'] = train_df['대출기간'].astype(int)

In [21]:
test_df.drop(['연체계좌수', '총연체금액'], axis=1, inplace=True)
test_df.loc[test_df['주택소유상태']=='ANY','주택소유상태'] = 'MORTGAGE'
test_df.loc[test_df['근로기간'] == '3', '근로기간'] = '3 years'
test_df.loc[test_df['근로기간'] == '1 year', '근로기간'] = '1 years'
test_df.loc[test_df['근로기간'] == '<1 year', '근로기간'] = '< 1 year'
test_df.loc[test_df['근로기간'] == '10+years', '근로기간'] = '10+ years'
test_df.loc[test_df['근로기간'] == 'Unknown', '근로기간'] = np.random.choice([1,5,10], size=sum(test_df['근로기간'] == 'Unknown'))
test_df['근로기간'] = test_df['근로기간'].replace(values_map)
test_df.loc[test_df['근로기간'] == 6, '근로기간'] = np.random.randint(1, 3)
test_df.loc[~test_df['대출목적'].isin(['부채 통합', '신용 카드', '기타']), '대출목적'] = '기타'
test_df.loc[test_df['대출기간']==' 36 months', '대출기간'] = 36
test_df.loc[test_df['대출기간']==' 60 months', '대출기간'] = 60
test_df['최근_2년간_연체_횟수'] = np.where(test_df['최근_2년간_연체_횟수'] > 2, 2, test_df['최근_2년간_연체_횟수'])
test_df['대출기간'] = test_df['대출기간'].astype(int)

In [58]:
train_data = train_df.copy()
test_data = test_df.copy()

In [59]:
train_data.drop(columns=['ID'], inplace=True)
test_data.drop(columns=['ID'], inplace=True)

In [60]:
train_y =train_data['대출등급']
train_X = train_data[train_data.columns.difference(['대출등급'])]
test_X = test_data[test_data.columns.difference(['대출등급'])]

In [61]:
train_X = pd.get_dummies(train_X)
test_X = pd.get_dummies(test_X)

전처리 완료, 모델 적용

In [62]:
train_X

,근로기간,대출금액,대출기간,부채_대비_소득_비율,연간소득,총계좌수,총상환원금,총상환이자,최근_2년간_연체_횟수,대출목적_기타,대출목적_부채 통합,대출목적_신용 카드,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT
0,5,12480000,36,18.90,72000000,15,0,0.0,0,0,1,0,0,0,1
1,10,14400000,60,22.33,130800000,21,373572,234060.0,0,1,0,0,1,0,0
2,1,12000000,36,8.60,96000000,14,928644,151944.0,0,0,1,0,1,0,0
3,5,14400000,36,15.09,132000000,15,325824,153108.0,0,0,1,0,1,0,0
4,5,18000000,60,25.39,71736000,19,228540,148956.0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,10,14400000,36,9.33,210000000,33,974580,492168.0,0,0,0,1,1,0,0
96290,10,28800000,60,5.16,132000000,25,583728,855084.0,0,1,0,0,1,0,0
96291,1,14400000,36,11.24,84000000,22,1489128,241236.0,0,0,0,1,1,0,0
96292,1,15600000,36,17.30,66330000,21,1378368,818076.0,2,0,1,0,1,0,0


In [63]:
test_X

,근로기간,대출금액,대출기간,부채_대비_소득_비율,연간소득,총계좌수,총상환원금,총상환이자,최근_2년간_연체_횟수,대출목적_기타,대출목적_부채 통합,대출목적_신용 카드,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT
0,5,16800000,36,19.64,132000000,12,394692,146604.0,0,1,0,0,1,0,0
1,1,8400000,36,15.84,89971200,25,0,0.0,0,0,1,0,0,0,1
2,5,17280000,36,8.41,150000000,20,1786980,281820.0,0,0,0,1,0,0,1
3,1,14400000,36,13.72,66000000,30,669024,281724.0,1,0,0,1,1,0,0
4,1,27600000,36,30.50,55200000,12,1250052,614844.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,1,30000000,36,22.08,78000000,27,1307532,763380.0,2,0,1,0,1,0,0
64193,10,30000000,60,12.06,109200000,26,960612,1245252.0,0,0,1,0,1,0,0
64194,10,6120000,36,28.80,39600000,33,131520,80880.0,0,0,1,0,0,0,1
64195,10,11520000,36,25.44,66000000,41,1339536,601872.0,1,0,1,0,1,0,0


방법 1. lgbm 적용 + 하이퍼파라미터 튜닝
방법 2. lgbm 적용한 다음 스태킹 앙상블
방법 3. cat, lgbm, rf 등등 소프트앙상블

In [64]:
import lightgbm as lgbm

lgbm_params = {
          'objective': 'multiclass',
          'num_class': 7,
          'metric': 'multi_logloss',
          'boosting_type': 'gbdt',
          'n_estimators': 15000,
          'num_leaves': 8,
          'min_child_samples': 12,
          'learning_rate': 0.15,
          'max_depth': 10,
          'subsample': 0.9,
          'colsample_bytree': 0.9,
          'subsample_freq': 1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.8,
          'verbose': -1}
lgbm_model = lgbm.LGBMClassifier(**lgbm_params)

In [65]:
# Fit the model to the training data
lgbm_model.fit(train_X, train_y)

# Make predictions on the test data
predictions = lgbm_model.predict(test_X)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
sub = pd.read_csv('sample_submission.csv')
sub['대출등급'] = predictions
sub.to_csv('drive/MyDrive/submission_lgbm.csv', index=False)

하이퍼파라미터 튜닝

In [28]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgbm
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, log_loss

X, y = train_X.sample(20000,random_state = 42), train_y.sample(20000,random_state = 42)

def macro_f1_score(y_true, preds):
    preds = preds.argmax(axis=1)  # Convert probability predictions to class labels
    return 'macro_f1', f1_score(y_true, preds, average='macro'), True

def objective(trial):
    params = {
        'objective': 'multiclass',
        'num_class': 7,
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 20000),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log = True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log = True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log = True),
        'verbose': -1
    }

    loglosses = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, valid_index in skf.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        model = lgbm.LGBMClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='multi_logloss')
        preds_proba = model.predict_proba(X_valid)
        logloss = log_loss(y_valid, preds_proba)
        loglosses.append(logloss)

    return np.mean(loglosses)


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params


[I 2024-02-02 07:23:20,930] A new study created in memory with name: no-name-289c2bf2-a14b-4e1b-9b72-77e8e8d802e7
[I 2024-02-02 07:23:31,738] Trial 0 finished with value: 0.6261849592194693 and parameters: {'n_estimators': 321, 'num_leaves': 11, 'min_child_samples': 99, 'learning_rate': 0.05291584307750763, 'max_depth': 8, 'subsample': 0.6455604312880946, 'colsample_bytree': 0.8287806859943065, 'subsample_freq': 6, 'reg_alpha': 0.0011192543372499785, 'reg_lambda': 0.7165000247283656}. Best is trial 0 with value: 0.6261849592194693.
[I 2024-02-02 07:32:46,823] Trial 1 finished with value: 0.8917336216244248 and parameters: {'n_estimators': 13993, 'num_leaves': 30, 'min_child_samples': 88, 'learning_rate': 0.08875436322083784, 'max_depth': 9, 'subsample': 0.6151531738770363, 'colsample_bytree': 0.814185902689216, 'subsample_freq': 1, 'reg_alpha': 0.12481698145925363, 'reg_lambda': 3.6779730428137754}. Best is trial 0 with value: 0.6261849592194693.
[I 2024-02-02 07:38:35,871] Trial 2 fin

In [ ]:
best_params_lgbm = study.best_params
best_params_lgbm

In [29]:
best_params = {'n_estimators': 2531,
 'num_leaves': 22,
 'min_child_samples': 19,
 'learning_rate': 0.011586835833334674,
 'max_depth': 15,
 'subsample': 0.9646775442982475,
 'colsample_bytree': 0.9086908650150887,
 'subsample_freq': 3,
 'reg_alpha': 0.20375236724138712,
 'reg_lambda': 0.013498879575478753}

In [ ]:
lgbm_params

{'objective': 'multiclass',
 'num_class': 6,
 'metric': 'multi_logloss',
 'boosting_type': 'gbdt',
 'n_estimators': 15000,
 'num_leaves': 8,
 'min_child_samples': 12,
 'learning_rate': 0.15,
 'max_depth': 10,
 'subsample': 0.9,
 'colsample_bytree': 0.9,
 'subsample_freq': 1,
 'reg_alpha': 0.1,
 'reg_lambda': 0.8,
 'verbose': -1}

In [ ]:
update_params = {'objective': 'multiclass',
 'num_class': 7,
 'metric': 'multi_logloss',
 'boosting_type': 'gbdt',
  'verbose' : -1}

best_params.update(update_params)

lgbm_model = lgbm.LGBMClassifier(**best_params)

lgbm_model.fit(train_X, train_y)

predictions = lgbm_model.predict(test_X)

sub = pd.read_csv('sample_submission.csv')
sub['대출등급'] = predictions
sub.to_csv('drive/MyDrive/submission_lgbmtuning.csv', index=False)

방법 2. nn을 이용한 스태킹 앙상블

In [ ]:
lgbm_model.predict(train_X)

In [ ]:
update_params = {'objective': 'multiclass',
 'num_class': 6,
 'metric': 'multi_logloss',
 'boosting_type': 'gbdt',
  'verbose' : -1}

best_params.update(update_params)

In [32]:
from sklearn.model_selection import StratifiedKFold
# Initialize Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X, y = train_X, train_y
# Initialize an array to store the predictions
predictions = np.zeros(len(y))
predictions = pd.Series(predictions)

# Loop through the folds
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the model on the training fold
    model = lgbm.LGBMClassifier(**best_params)
    model.fit(X_train, y_train)

    # Predict the test fold
    fold_predictions = model.predict(X_test)

    # Store predictions for the test fold
    predictions[test_index] = fold_predictions


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1394
[LightGBM] [Info] Number of data points in the train set: 77035, number of used features: 15
[LightGBM] [Info] Start training from score -1.747737
[LightGBM] [Info] Start training from score -1.206421
[LightGBM] [Info] Start training from score -1.248773
[LightGBM] [Info] Start training from score -1.975606
[LightGBM] [Info] Start training from score -2.572193
[LightGBM] [Info] Start training from score -3.897013
[LightGBM] [Info] Start training from score -5.434904
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001010 seconds.
You c

In [35]:
sub = pd.read_csv('sample_submission.csv')
sub['대출등급'] = predictions
sub.to_csv('drive/MyDrive/submission_5fold_lgbm.csv', index=False)

In [ ]:
predictions

In [34]:
train_X

,근로기간,대출금액,대출기간,부채_대비_소득_비율,연간소득,총계좌수,총상환원금,총상환이자,최근_2년간_연체_횟수,대출목적_기타,대출목적_부채 통합,대출목적_신용 카드,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT,lgbm_predictions
0,5,12480000,36,18.90,72000000,15,0,0.0,0,0,1,0,0,0,1,B
1,10,14400000,60,22.33,130800000,21,373572,234060.0,0,1,0,0,1,0,0,B
2,1,12000000,36,8.60,96000000,14,928644,151944.0,0,0,1,0,1,0,0,A
3,5,14400000,36,15.09,132000000,15,325824,153108.0,0,0,1,0,1,0,0,C
4,5,18000000,60,25.39,71736000,19,228540,148956.0,0,1,0,0,0,0,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,10,14400000,36,9.33,210000000,33,974580,492168.0,0,0,0,1,1,0,0,C
96290,10,28800000,60,5.16,132000000,25,583728,855084.0,0,1,0,0,1,0,0,E
96291,1,14400000,36,11.24,84000000,22,1489128,241236.0,0,0,0,1,1,0,0,A
96292,1,15600000,36,17.30,66330000,21,1378368,818076.0,2,0,1,0,1,0,0,D


In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

label_encoder = LabelEncoder()
train_X['lgbm_predictions'] = label_encoder.fit_transform(train_X['lgbm_predictions'])

# Define your data
X_train,X_val,y_train,y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

# Encode labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
num_classes = len(label_encoder.classes_)

def create_model(trial):
    # Define hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 1, 5)
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    num_units = trial.suggest_int('num_units', 32, 512)

    # Create the model
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train.shape[1:]))

    for i in range(num_layers):
        model.add(keras.layers.Dense(units=num_units, activation=activation))
        model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    # Clear previous session
    tf.keras.backend.clear_session()

    # Create model
    model = create_model(trial)

    # Train the model
    model.fit(X_train, y_train_encoded, validation_data=(X_val, y_val_encoded), epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    _, val_accuracy = model.evaluate(X_val, y_val_encoded, verbose=0)

    return val_accuracy

# Optimize hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Create and train the final model with the best hyperparameters
best_model = create_model(study.best_trial)
best_model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, verbose=0)

# Evaluate the final model
_, test_accuracy = best_model.evaluate(X_val, y_val_encoded)
print("Test Accuracy:", test_accuracy)


[I 2024-02-01 14:40:17,414] A new study created in memory with name: no-name-72004ead-dbee-45bc-bb68-16a9c6a03ff7
<ipython-input-57-1172dd5aa6b7>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-57-1172dd5aa6b7>:25: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
[I 2024-02-01 14:41:56,876] Trial 0 finished with value: 0.28526923060417175 and parameters: {'num_layers': 3, 'activation': 'relu', 'learning_rate': 1.3734260937442744e-05, 'dropout_rate': 0.36472830221180813, 'num_units': 249}. Best is trial 0 with value: 0.28526923060417175.
<ipyth

Best hyperparameters: {'num_layers': 2, 'activation': 'tanh', 'learning_rate': 2.825276352254286e-05, 'dropout_rate': 0.2697824448692781, 'num_units': 257}
602/602 [==============================] - 1s 1ms/step - loss: 1.5400 - accuracy: 0.3366
Test Accuracy: 0.3365699052810669


In [ ]:
# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

Best hyperparameters: {'num_layers': 2, 'activation': 'tanh', 'learning_rate': 2.825276352254286e-05, 'dropout_rate': 0.2697824448692781, 'num_units': 257}


In [37]:
best_params = {'num_layers': 2, 'activation': 'tanh', 'learning_rate': 2.825276352254286e-05, 'dropout_rate': 0.2697824448692781, 'num_units': 257}

In [46]:
def create_model(best_params):
    # Extract parameters from best_params
    num_layers = best_params['num_layers']
    activation = best_params['activation']
    learning_rate = best_params['learning_rate']
    dropout_rate = best_params['dropout_rate']
    num_units = best_params['num_units']

    # Create the model
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=train_X.shape[1:]))

    for i in range(num_layers):
        model.add(keras.layers.Dense(units=num_units, activation=activation))
        model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(6, activation='softmax'))

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [39]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

label_encoder = LabelEncoder()
train_X['lgbm_predictions'] = label_encoder.fit_transform(train_X['lgbm_predictions'])
train_y = label_encoder.transform(train_y)

In [54]:
np.unique(train_X['lgbm_predictions'])

array([0, 1, 2, 3, 4, 5, 6])

In [47]:
best_model = create_model(best_params)
best_model.fit(train_X, train_y, epochs=10, batch_size=32, verbose=0)

InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-

In [ ]:
train_X.to_csv('drive/MyDrtrain_X.csv')